In [50]:
#!pip install pandas

In [51]:
#!pip install pydub
#!pip install librosa
#!pip install ffmpeg
#!pip install ffprobe

# Carnatic Music: Recognizing Ragas

In the Indian Classical Music system of Carnatic Music, every piece that is rendered is usually in a raga. A raga is a sequence or pattern of notes chosen from 12 possible notes. Sometimes, two ragas can have the same exact notes but the strict pattern of the scale going up or down can differentiate them. 

In addition, two ragas can be different by the change of just one or two notes. 
The adherence to a raga in a piece is very strict in Carnatic music, and it is often an exercise for the listener to recognize the raga that is being rendered. 

After years of listening to and learning Carnatic Music vocally and instrumentally, I have practiced this exercise many times. If I could recognize a ragam based off of what I hear, could an algorithm do the same based off of patterns in audio files?

In order to explore this question, I decided to do some data analysis and use popular Machine Learning Algorithms to see if an algorithm can recognize a raga. 

In order to simplify the project, I decided to use audio recordings of the 7 different pieces, each one rendered by 6 musicians each. I cut each audio into 30 second chunks (since this is the time it would take a novice to recognize a raga), extracted mean MFCC from each of them (MFCC will be explained later), and ran different Machine Learning Algorithms on the MFCC. 


In [2]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import librosa
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

## Pre-processing

Below is the code to cut each audio file into 30 second segments

In [3]:
#this cell splits each audio file into 30 second segments
from pydub import AudioSegment

def split(songs, raga):
#     song = AudioSegment.from_file(file, format="mp3")
    start = 0
    for s in songs:
        song = AudioSegment.from_file(s, format="mp3")
        for i, chunk in enumerate(song[::30000]):
            
            with open("segments/%s/%s.mp3" % (raga, i + start), "wb") as f:
                chunk.export(f, format="mp3")
        start = start + i
    print("done")
        
abhogi = ["carnatic_varnam_1.0/Audio/223578__gopalkoduri__carnatic-varnam-by-dharini-in-abhogi-raaga.mp3", 
         "carnatic_varnam_1.0/Audio/223579__gopalkoduri__carnatic-varnam-by-prasanna-in-abhogi-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223580__gopalkoduri__carnatic-varnam-by-ramakrishnamurthy-in-abhogi-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223581__gopalkoduri__carnatic-varnam-by-sreevidya-in-abhogi-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223582__gopalkoduri__carnatic-varnam-by-vignesh-in-abhogi-raaga.mp3"]

begada = ["carnatic_varnam_1.0/Audio/223583__gopalkoduri__carnatic-varnam-by-prasanna-in-begada-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223584__gopalkoduri__carnatic-varnam-by-sreevidya-in-begada-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223585__gopalkoduri__carnatic-varnam-by-vignesh-in-begada-raaga.mp3",
         ]

kalyani = ["carnatic_varnam_1.0/Audio/223586__gopalkoduri__carnatic-varnam-by-badrinarayanan-in-kalyani-raaga.mp3",
          "carnatic_varnam_1.0/Audio/223587__gopalkoduri__carnatic-varnam-by-prasanna-in-kalyani-raaga.mp3",
          "carnatic_varnam_1.0/Audio/223588__gopalkoduri__carnatic-varnam-by-ramakrishnamurthy-in-kalyani-raaga.mp3",
          "carnatic_varnam_1.0/Audio/223589__gopalkoduri__carnatic-varnam-by-vignesh-in-kalyani-raaga.mp3"]

mohanam = ["carnatic_varnam_1.0/Audio/223590__gopalkoduri__carnatic-varnam-by-badrinarayanan-in-mohanam-raaga.mp3",
          "carnatic_varnam_1.0/Audio/223591__gopalkoduri__carnatic-varnam-by-dharini-in-mohanam-raaga.mp3",
          "carnatic_varnam_1.0/Audio/223592__gopalkoduri__carnatic-varnam-by-prasanna-in-mohanam-raaga.mp3",
          "carnatic_varnam_1.0/Audio/223593__gopalkoduri__carnatic-varnam-by-ramakrishnamurthy-in-mohanam-raaga.mp3"]

sahana = ["carnatic_varnam_1.0/Audio/223594__gopalkoduri__carnatic-varnam-by-dharini-in-sahana-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223595__gopalkoduri__carnatic-varnam-by-ramakrishnamurthy-in-sahana-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223596__gopalkoduri__carnatic-varnam-by-sreevidya-in-sahana-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223597__gopalkoduri__carnatic-varnam-by-vignesh-in-sahana-raaga.mp3"]

saveri = ["carnatic_varnam_1.0/Audio/223598__gopalkoduri__carnatic-varnam-by-badrinarayanan-in-saveri-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223599__gopalkoduri__carnatic-varnam-by-dharini-in-saveri-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223600__gopalkoduri__carnatic-varnam-by-prasanna-in-saveri-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223601__gopalkoduri__carnatic-varnam-by-ramakrishnamurthy-in-saveri-raaga.mp3",
         "carnatic_varnam_1.0/Audio/223602__gopalkoduri__carnatic-varnam-by-sreevidya-in-saveri-raaga.mp3"]

sri = ["carnatic_varnam_1.0/Audio/223603__gopalkoduri__carnatic-varnam-by-badrinarayanan-in-sri-raaga.mp3",
      "carnatic_varnam_1.0/Audio/223604__gopalkoduri__carnatic-varnam-by-ramakrishnamurthy-in-sri-raaga.mp3",
      "carnatic_varnam_1.0/Audio/223605__gopalkoduri__carnatic-varnam-by-vignesh-in-sri-raaga.mp3"]


# split(abhogi, "Abhogi") 
# split(begada, "Begada")
# split(kalyani, "Kalyani")
# split(mohanam, "Mohanam")
# split(sahana, "Sahana")
# split(saveri, "Saveri")
# split(sri, "Sri")

### About the Given Data

For each ragam, after splitting into the 30 second segments, there are about 60 segments. So, in total there are about 420 records, since there are 7 ragas that we are going to be classifying. 

### MFCC Constants

The code below extracts the MFCC constants from each of the audio files. 
What is an MFCC constant?
MFCC are the Mel Frequency Cepstral Constants, which are computed at equally spaced times at a constant sampling period. MFCCs start by taking the Fourier Transform of a signal, then mapping the powers of the spectrum onto a mel scale (a logarithmic transformation of a signal's frequency - this is often done to mimic a human's recognition of pitch and sound. Usually, a human will more easily recognize the difference between two pitches that are closer together rather than two pitches that are far apart. For the data to mimic this idea, we do use a logarithmic transformation). Then, the discrete cosine transform of the mel log powers are taken, and the mfccs end up being the amplitudes of this resulting spectrum.  

I ended up obtaining a list of lists for each audio file of 30 seconds - the inner lists being the mfcc constants for each window smaller window withing the 30 second audio. Since there were many such windows, I thought it a good idea to take the average of each of these windows to summarize what was happening at each segment of the 30 second audio. This data was stored in a csv file with the particular raga that it corresponded to. 


In [4]:


warnings.filterwarnings("ignore")

def mfcc_per_raga(file_path):
    
    directory = file_path
    mfccs = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename) 
        if os.path.isfile(f):

            data, sampling_rate = librosa.load(f)
            mfcc = librosa.feature.mfcc(y=data, sr=sampling_rate)
            mfccs.append(mfcc)
            
    return mfccs

# abhogi = mfcc_per_raga("segments/Abhogi") # 1
# begada = mfcc_per_raga("segments/Begada") # 2
# kalyani = mfcc_per_raga("segments/Kalyani") # 3
# mohanam = mfcc_per_raga("segments/Mohanam") # 4
# sahana = mfcc_per_raga("segments/Sahana") # 5
# saveri = mfcc_per_raga("segments/Saveri") # 6
# sri = mfcc_per_raga("segments/Sri") # 7


In [5]:

def average(lst):
    return sum(lst)/len(lst)


The code below is extracting the mfcc constants from each audio file and averaging the constants for each smaller window (the mfcc algorithm by default goes by windows).

In [6]:


def prepare_data(raga_list, y_vals, x_vals, raga_number):
    for clip in raga_list:
        clips = []
        for mfccs in clip:
            clips.append(average(mfccs))
        y_vals.append(raga_number)
        x_vals.append(clips)
    return x_vals, y_vals

# x_vals = []
# y_vals = []

# x_vals, y_vals = prepare_data(abhogi, y_vals, x_vals, 1)
# x_vals, y_vals = prepare_data(begada, y_vals, x_vals, 2)
# x_vals, y_vals = prepare_data(kalyani, y_vals, x_vals, 3)
# x_vals, y_vals = prepare_data(mohanam, y_vals, x_vals, 4)
# x_vals, y_vals = prepare_data(sahana, y_vals, x_vals, 5)
# x_vals, y_vals = prepare_data(saveri, y_vals, x_vals, 6)
# x_vals, y_vals = prepare_data(sri, y_vals, x_vals, 7)




# data = x_vals
# df = pd.DataFrame(data)
# df['y_val'] = y_vals


# df = pd.read_csv('saved_data.csv')
# print(df)

I have already saved the data in a separate file, so I make a data frame out of it. 

In [7]:
#jumble up the data
# df_shuffled=df.sample(frac=1).reset_index(drop=True)
df_shuffled = pd.read_csv('shuffled_data.csv')

In [68]:

df = pd.read_csv('saved_data.csv')


I split the data into training and test sets, using the first 75% of the jumbled data set as the training and the last 25% as the test. I also use cross-validation methods (which is shown later).

In [9]:
# splitting data

X = df_shuffled.iloc[:,:-1]
Y = df_shuffled.iloc[:, -1]

X_train = X.head(291)
Y_train = Y.head(291)

X_test = X.tail(96)
Y_test = Y.tail(96)



## Decision Tree

I decided to try a decision tree because of the very subtle details between the frequencies in the different ragas. In addition, since some of the frequencies may get slightly masked due to the averaging over windows of time, the data becomes even more subtle. For this reason, I thought it would be a good idea to use a decision tree since it will make categories based off of number bounds in the data, which get smaller and more specific as we get down the tree.


In [28]:
#Decision Tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)
scores = cross_val_score(clf, X, Y, cv=5)
print("Accuracy with cross validation: ", np.average(scores))
labels = [1, 2, 3, 4, 5, 6, 7]
conf1 = confusion_matrix(Y_test, Y_pred, labels=labels) #figure out what to do with "labels"

print(conf1)

Accuracy with cross validation:  0.6333000333000333
[[11  0  0  0  0  0  1]
 [ 0 10  0  0  0  0  0]
 [ 0  1 12  0  2  1  0]
 [ 0  0  0 10  0  0  0]
 [ 2  0  0  0 15  0  1]
 [ 2  1  0  0  0 11  1]
 [ 0  0  0  0  0  0 15]]


This decision tree seems to work fairly well, with around a 61% accuracy. I thought it would be a good idea to use cross validation too to see how the model works across different windows of data. It performed slightly better with cross validation

## Random Forest

Since the Decision Tree seemed to work well, I thought that a combination of decision trees may work better. For that reason, I implemented a random forest. 

In [22]:
#Random Forest
rfc = RandomForestClassifier()
rfc.fit(X_train, Y_train)
scores = cross_val_score(rfc, X, Y, cv=5)
print("Accuracy with cross validation: ", np.average(scores))
conf2 = confusion_matrix(Y_test, Y_pred)
print(conf2)

Accuracy with cross validation:  0.8630036630036629
[[11  0  0  0  0  0  1]
 [ 0 10  0  0  0  0  0]
 [ 0  0 13  0  1  2  0]
 [ 1  0  0  9  0  0  0]
 [ 2  0  1  0 15  0  0]
 [ 2  0  0  0  0 12  1]
 [ 0  0  0  0  0  0 15]]


The Random Forest seems to come up with a much higher accuracy than the decision tree. It seems to work well! 

In [30]:

abc = AdaBoostClassifier(base_estimator=clf)
print(abc)
preds = cross_val_predict(abc, X, Y, cv=5)
print("AdaBoost accuracy:", accuracy_score(Y, preds))
conf3 = confusion_matrix(Y, preds)
print(conf3)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
AdaBoost accuracy: 0.6589147286821705
[[36  0  4  7  5  2  3]
 [ 6 37  3  1  2  2  1]
 [ 1  1 35  0  8  6  2]
 [ 5  2  3 32  0  4  2]
 [ 5  4  4  4 32  4  3]
 [ 4  2  7  3  1 48  4]
 [ 3  1  1  5  3  4 35]]


The accuracy for using AdaBoost on a decision tree is slightly larger than the accuracy of just a decision tree. 

In [24]:
abc = AdaBoostClassifier(base_estimator=rfc)
print(abc)
preds = cross_val_predict(abc, X, Y, cv=5)
print("AdaBoost accuracy:", accuracy_score(Y, preds))
conf3 = confusion_matrix(Y, preds)
print(conf3)

AdaBoostClassifier(base_estimator=RandomForestClassifier())
AdaBoost accuracy: 0.8837209302325582
[[45  0  2  2  5  0  3]
 [ 1 48  1  0  0  2  0]
 [ 3  0 46  0  1  3  0]
 [ 4  0  0 43  0  0  1]
 [ 2  0  2  0 51  0  1]
 [ 2  0  3  0  2 59  3]
 [ 0  1  0  0  1  0 50]]


The accuracy for using AdaBoost on a random forest is slightly larger than the accuracy of just the random forest.

In [67]:
#Neural Network
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(20, 18, 15), random_state=1, activation='relu', max_iter=500)
clf.fit(X_train, Y_train)
preds = cross_val_predict(clf, X, Y, cv=5)
print("NN accuracy:", accuracy_score(Y, preds))
conf4 = confusion_matrix(Y, preds)
print(conf4)

NN accuracy: 0.49870801033591733
[[24  4  3  5  8  5  8]
 [ 2 28  1  6  6  6  3]
 [ 5  3 21  2  1 19  2]
 [ 7  1  1 21  4  5  9]
 [ 4  8  2  1 30  2  9]
 [ 1 13  3  8  2 36  6]
 [ 7  0  2  1  4  5 33]]


I have tried different combinations of hidden layer sizes, activation functions, and solvers. Out of all the combinations, this had the highest accuracy. It is clear that a Neural Network isn't able to pick up the nuances in ragas as well as decision trees. 

# Conclusion

From the analysis above, I can conclude that a Random Forest is the best model to classify ragas. This also told me a lot about the nature of the data - since only Random Forests and Decision trees seemed to give the best accuracy. These algorithms work by splitting the data meticulously based on very specific intervals, which was needed for the complex data that I worked with. 

In order to improve the accuracy, I could also work with more data - more audio files and also find a different method for pre-processing, perhaps keeping original values instead of taking MFCC means (which I did to slightly simplify the project). I could also use different songs in each raga rather than the same composition to test out how well my model works. 

I only worked with seven different ragas in this project, all of which are pretty distrinct in terms of patterns. The next step would be working with ragas that are much closer to each other. 